In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# test reading data

import json
from sklearn.model_selection import train_test_split

with open("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/data/geonames_train_curated.json", "r") as json_data:
    geonames_json = json.loads(json_data.read())
    json_data.close()

print(geonames_json[0])

# transform json data to pandas dataframe
#wordnet_df = pd.DataFrame.from_dict(wordnet_json)
#print(wordnet_df.head())

# train-test split data
geonames_train, geonames_test = train_test_split(geonames_json, test_size=0.3)

{'ID': 1390498, 'term': 'Yilingsantuan Qixiangzhan', 'type': 'meander neck'}


In [3]:
# start preparing for QA pipeline
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
# prepare train dataset
geonames_train_data = []
for i in geonames_train:
  geonames_train_data.append(i)


# prepare test dataset
geonames_test_data = []
for i in geonames_test:
  geonames_test_data.append(i)

In [5]:
!pip install transformers
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [6]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

In [7]:
MODEL_NAME = "google/flan-t5-small"
TOKENIZER = T5TokenizerFast.from_pretrained(MODEL_NAME)
MODEL = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 512   # Question Length
T_LEN = 512    # Target Length
BATCH_SIZE = 4
DEVICE = "cuda:0"

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
# Create a Dataframe from train data
geonames_train_df = pd.DataFrame(geonames_train_data)
# convert df values to string
geonames_train_df = geonames_train_df.applymap(str)

# Create a Dataframe from test data
geonames_test_df = pd.DataFrame(geonames_test_data)
# convert df values to string
geonames_test_df = geonames_test_df.applymap(str)

In [9]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.term = self.data["term"]
        self.ttype = self.data['type']

    def __len__(self):
        return len(self.term)

    def __getitem__(self, idx):
        term = self.term[idx]
        ttype = self.ttype[idx]

        term_tokenized = self.tokenizer(term, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        type_tokenized = self.tokenizer(ttype, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(type_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(term_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(term_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(type_tokenized["attention_mask"], dtype=torch.long)
        }

In [10]:
geo_train_dataset = QA_Dataset(TOKENIZER, geonames_train_df, Q_LEN, T_LEN)
geo_test_dataset = QA_Dataset(TOKENIZER, geonames_test_df, Q_LEN, T_LEN)

train_loader = DataLoader(geo_train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(geo_test_dataset, batch_size=BATCH_SIZE)

In [11]:
#torch.cuda.empty_cache()

MODEL.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [12]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(5):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

    print(f"{epoch+1}/{5} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Validation batches: 100%|██████████| 3726/3726 [05:59<00:00, 10.35it/s]


1/5 -> Train loss: 2.6223143995953255	Validation loss: 1.5223337312259473


Validation batches: 100%|██████████| 3726/3726 [05:59<00:00, 10.35it/s]


2/5 -> Train loss: 2.1430674575720903	Validation loss: 1.3764330500947293


Validation batches: 100%|██████████| 3726/3726 [06:00<00:00, 10.34it/s]


3/5 -> Train loss: 1.9100549971985006	Validation loss: 1.2825632864115455


Validation batches: 100%|██████████| 3726/3726 [05:59<00:00, 10.36it/s]


4/5 -> Train loss: 1.7642926144456825	Validation loss: 1.2102012719317392


Validation batches: 100%|██████████| 3726/3726 [05:59<00:00, 10.36it/s]

5/5 -> Train loss: 1.660701752813501	Validation loss: 1.1488701219149573


In [13]:
MODEL.save_pretrained("flan-t5-small_geonames_model")
TOKENIZER.save_pretrained("flan-t5-small_geonames_tokenizer")

('flan-t5-small_geonames_tokenizer/tokenizer_config.json',
 'flan-t5-small_geonames_tokenizer/special_tokens_map.json',
 'flan-t5-small_geonames_tokenizer/spiece.model',
 'flan-t5-small_geonames_tokenizer/added_tokens.json',
 'flan-t5-small_geonames_tokenizer/tokenizer.json')

In [14]:
# Saved files
"""('flan-t5-small_geonames_tokenizer/tokenizer_config.json',
 'flan-t5-small_geonames_tokenizer/special_tokens_map.json',
 'flan-t5-small_geonames_tokenizer/spiece.model',
'flan-t5-small_geonames_tokenizer/added_tokens.json',
'flan-t5-small_geonames_tokenizer/tokenizer.json')"""

"('flan-t5-small_geonames_tokenizer/tokenizer_config.json',\n 'flan-t5-small_geonames_tokenizer/special_tokens_map.json',\n 'flan-t5-small_geonames_tokenizer/spiece.model',\n'flan-t5-small_geonames_tokenizer/added_tokens.json',\n'flan-t5-small_geonames_tokenizer/tokenizer.json')"

In [15]:
import numpy as np

def precision_at_k(actual, predicted):
    act_set = set(actual)
    pred_set = set(predicted)
    result = len(act_set & pred_set) / float(len(predicted))
    return result*100

def apk(actual, predicted, k):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if not actual:
        return 0.0
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / min(len(actual), k)


def mapk(actual, predicted, k):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int,
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a, p, k) for a,p in zip(actual, predicted)])


class EvaluationMetrics:

    def __init__(self, ks:list, metric="map") -> None:
        self.ks = ks
        self.metric=metric

    def evaluate(self, actual:list, predicted:list):
        if self.metric == "map":
            return self.MAP(actual, predicted)
        else:
            return self.AP(actual, predicted)

    def MAP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["MAP@"+str(k)] = mapk(actual=actual, predicted=predicted, k=k)
        return results_dict

    def AP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["AP@"+str(k)] = [apk(actual=actual, predicted=predicted, k=k)
                                          for a, p in zip(actual, predicted)]
        return results_dict

def predict_answer(term, ref_type=None):
    inputs = TOKENIZER(term, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_type = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_type:
        # Load the MAP@K metric for K=1
        score = mapk([ref_type], [predicted_type], 1)

        print("Term: \n", term)
        return {
            "Reference Type: ": ref_type,
            "Predicted Type: ": predicted_type,
            "MAP@1: ": score
        }
    else:
        return predicted_type

In [16]:
# test predictions
term = geonames_train_df.iloc[0]["term"]
ttype = geonames_train_df.iloc[0]["type"]

print("*** PREDICTION:", predict_answer(term, ttype))

Term: 
 Woldegker Heide
*** PREDICTION: {'Reference Type: ': 'heath', 'Predicted Type: ': 'heath', 'MAP@1: ': 1.0}


In [20]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

# test loading model
MODEL = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/model/flan-t5-small_geonames_model")
TOKENIZER = T5TokenizerFast.from_pretrained("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/model/flan-t5-small_geonames_tokenizer")

In [21]:
MODEL.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [22]:
# test predictions
reference_types = []
predicted_types = []
for index, row in geonames_test_df.iterrows():
  term = row["term"]
  ttype = row["type"]

  predicted_type = predict_answer(term, ttype)["Predicted Type: "]
  reference_types.append(ttype)
  predicted_types.append(predicted_type)

print("MAP@1 score:", mapk(reference_types, predicted_types, 1))

Streaming output truncated to the last 5000 lines.
 Buckley Bay
Term: 
 City Glasses Watergate
Term: 
 Sar-e Yort
Term: 
 Hofmann Trough
Term: 
 Barrio Privado Aldea
Term: 
 Lichu
Term: 
 Inspection Bungalow Daman
Term: 
 Khor Qowab
Term: 
 Ab Bandan-e Valuja
Term: 
 Sesil Isthmus
Term: 
 Pas de la Casa
Term: 
 Khabb al Fu'ad
Term: 
 Wardercarrin Soak
Term: 
 Serrat del Caire Forc
Term: 
 Pagano Nunatak
Term: 
 Salina Las Barrancas
Term: 
 Getikvank'
Term: 
 Shkoder Mediumwave Transmitter
Term: 
 Papendorp
Term: 
 Elato Atoll
Term: 
 Saint-Liboire
Term: 
 Challenger Harbour
Term: 
 Hamriya Free Zone Port
Term: 
 Podok-kak
Term: 
 Pishan Keyunzhan
Term: 
 Rikujo Jieitai Makomanai Chutonchi Shagekijo
Term: 
 Cuesta Rapel
Term: 
 Huttental
Term: 
 Aduana las Lajas
Term: 
 Sudonapseokgwangsan
Term: 
 Plantation Park at Ballantyne Apartments
Term: 
 Shureh Zar-e Jalakeh
Term: 
 Wetterkreuze
Term: 
 Lurio
Term: 
 Andzavadzor
Term: 
 Cuesta Nacascolo
Term: 
 Birkat al Ghalyun
Term: 
 Gamen Re

In [17]:
# export test data split to re-test on full RAG pipeline
geonames_test_df.to_csv("geonames_test_df.csv", index=False)